In [ ]:
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score, GridSearchCV
from scipy.cluster.hierarchy import dendrogram, fcluster, fclusterdata, linkage
from sklearn.neighbors import kneighbors_graph
import scipy.cluster.hierarchy as shc
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
import scipy.spatial as spatial
from sklearn.metrics import pairwise_distances
from collections import Counter
import matplotlib.pyplot as plt
from timeit import default_timer as timer
import numpy as np
from sklearn.metrics import accuracy_score
from datetime import timedelta
import stablerank_custom.srank as sr
from scipy.sparse import csgraph
inf = float("inf")
nan = np.nan

In [ ]:
import geopandas as gpd
deso_url = 'https://geodata.scb.se/geoserver/stat/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=stat%3ADeSO.2018&outputFormat=Geopackage&format_options=charset:UTF-8'
deso = gpd.read_file(deso_url)
deso["x"] = deso.centroid.map(lambda p: p.x)
deso["y"] = deso.centroid.map(lambda p: p.y)

In [ ]:
df = pd.read_pickle('output/enriched_data.pkl')
#df = df.merge(deso[['x', 'y', 'deso']], left_on = 'deso', right_on = 'deso')
df['TEAM_MAGDA'] = df[['S', 'V', 'C', 'MP']].sum(axis = 1)
df['TEAM_UFFE'] = df[['M', 'SD', 'L', 'KD']].sum(axis = 1)
for p in ['S', 'SD', 'M', 'V', 'C', 'KD', 'MP', 'L']:
    df[p + '_'] = df[p] - df[p].mean()
df['STÖRSTA_PARTI_ADJUSTED'] = df[['S_', 'SD_', 'M_', 'V_', 'C_', 'KD_', 'MP_', 'L_']].idxmax(axis=1)
df['STÖRSTA_BLOCK'] = df[['TEAM_MAGDA', 'TEAM_UFFE']].idxmax(axis=1)
df['ORTSTYP'] = df.deso.str[4]
scaler = StandardScaler()
non_model_variables = ['deso',
                        'kommun', 'kommunnamn', 'lan',
                        'index_right', 'Lkfv', 'Vdnamn',
                        'Distriktkod', 'Distriktnamn', 'ORTSTYP',
                        'S', 'SD', 'M', 'V', 'C', 'KD', 'MP', 'L',
                        'S_', 'SD_', 'M_', 'V_', 'C_', 'KD_', 'MP_', 'L_',
                        'STÖRSTA_PARTI', 'STÖRSTA_PARTI_ADJUSTED',
                        'TEAM_MAGDA', 'TEAM_UFFE']
#non_model_variables = ['deso',
#                        'kommun', 'lannamn', 'lan',
#                        'index_right', 'Lkfv', 'Vdnamn',
#                        'Distriktkod', 'Distriktnamn', 'ORTSTYP',
#                        'S', 'SD', 'M', 'V', 'C', 'KD', 'MP', 'L',
#                        'S_', 'SD_', 'M_', 'V_', 'C_', 'KD_', 'MP_', 'L_',
#                        'STÖRSTA_PARTI', 'STÖRSTA_PARTI_ADJUSTED',
#                        'TEAM_MAGDA', 'TEAM_UFFE']
y = df['STÖRSTA_BLOCK']
#df.set_index(['STÖRSTA_BLOCK', 'kommunnamn'], inplace = True)
df.set_index(['STÖRSTA_BLOCK', 'lannamn'], inplace = True)
X = df.drop(non_model_variables, axis = 1)
X = pd.DataFrame(data = scaler.fit_transform(X), index = X.index, columns = X.columns)
#X['x'] = deso['x'].values
#X['y'] = deso['y'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)

In [ ]:
# Fitting SVM for reference model
param_grid = {
    "pca__n_components": [0.5, 0.6, 0.7, 0.8, 0.9, 0.99],
    "svc__C": np.linspace(0.1,10,20),
    "svc__kernel": ['rbf',],
    "svc__gamma": ['auto'],
}
svc = SVC()
pca = PCA()
pipe = Pipeline([('pca', pca), ('svc',svc)])
search = GridSearchCV(pipe, param_grid, n_jobs=2, verbose = 1)
search.fit(X_train, y_train)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)
y_hat = search.predict(X_test)
counter = 0
for i,j in zip(y_test.values, y_hat):
    if i == j:
        counter += 1
    else:
        pass

In [ ]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)
print(counter/len(y_test))

In [ ]:
clustering_methods = ['single', 'average', 'complete', 'ward']
distance = 'cityblock'

In [ ]:
# Converitng the data into distance objects
train_dist = [sr.Distance(spatial.distance.pdist(X_train.loc[X_train.index == i], distance)) for i in X_train.index.unique()]
test_dist = [sr.Distance(spatial.distance.pdist(X_test.loc[X_test.index == i], distance)) for i in X_test.index.unique()]

In [ ]:
# Converitng the distance objects into H0 stable ranks
h0sr_train = {}
h0sr_test = {}
for cm in clustering_methods:
    h0sr_train[cm] = [d.get_h0sr(clustering_method=cm) for d in train_dist]
    h0sr_test[cm] = [d.get_h0sr(clustering_method=cm) for d in test_dist]

In [ ]:
# Converitng the distance objects into bar_codes and stable ranks
bc_train = [d.get_bc(maxdim=1) for d in  train_dist]
bc_test = [d.get_bc(maxdim=1) for d in  test_dist]
h1sr_train = [sr.bc_to_sr(bar_code, degree="H1") for bar_code in bc_train]
h1sr_test = [sr.bc_to_sr(bar_code, degree="H1") for bar_code in bc_test]

In [ ]:
h0sr_train = {}
h0_kernel_train ={}
h1sr_train = []
for cm in clustering_methods:
    h0sr_train[cm] = []
    for i in X_train.index.unique():
        X_ = X_train.loc[X_train.index == i]
        indicator = False
        j = 1
        while indicator == False:
            knn_graph = kneighbors_graph(X_, j, include_self = False, metric = metric)
            n_components, labels = csgraph.connected_components(knn_graph)
            if (n_components == 1):
                indicator = True
            else:
                j += 1
        model = AgglomerativeClustering(n_clusters=len(X_), connectivity = knn_graph, compute_full_tree = True, linkage = cm, affinity = 'euclidean', compute_distances = True)
        model.fit(X_[['x', 'y']])
        Z = np.transpose(np.array([model.children_[:,0], model.children_[:,1], model.distances_]))
        #print('Computing h0', cm, i, j, len(X_))
        h0sr = sr._linkage_to_stable_rank(Z, contour = sr.standard_contour(), w_p = inf, w_q = 1, reduced = True)
        h0sr_train[cm].append([h0sr, i[0]])
        # Need to encode the knn connectivity into d_
        knn_array = knn_graph.toarray()
        np.fill_diagonal(knn_array, 1)
        distance_full = spatial.distance.pdist(X_[['x', 'y']], "euclidean")
        distance_ref = np.max(distance_full)
        d_ = spatial.distance.squareform(distance_full)
        d_[np.where(knn_array == 0)] = distance_ref
        
        bc = sr._d_to_bc(d_, maxdim=1, thresh=inf, coeff=2, reduced=True)
        if len(h1sr_train) < len(X_train.index.unique()):
            #print('Computing h1')
            h1sr = sr.bc_to_sr(bc, degree="H1")
            h1sr_train.append([h1sr, i[0]])
        # get bc (how? just send distance matrix?)
        # get h1sr from bc
    h0_kernel_train[cm] = np.asarray([[f[0].dot(g[0]) for g in h0sr_train[cm]] for f in h0sr_train[cm]])
h1_kernel_train = np.asarray([[f[0].dot(g[0]) for g in h1sr_train] for f in h1sr_train])

In [ ]:
h0sr_test = {}
h0_kernel_test ={}
h1sr_test = []
for cm in clustering_methods:
    h0sr_test[cm] = []
    for i in X_test.index.unique():
        X_ = X_test.loc[X_test.index == i]
        indicator = False
        j = 1
        while indicator == False:
            knn_graph = kneighbors_graph(X_, j, include_self = False, metric = metric)
            n_components, labels = csgraph.connected_components(knn_graph)
            if (n_components == 1):
                indicator = True
            else:
                j += 1
        model = AgglomerativeClustering(n_clusters=len(X_), connectivity = knn_graph, compute_full_tree = True, linkage = cm, affinity = 'euclidean', compute_distances = True)
        model.fit(X_[['x', 'y']])
        Z = np.transpose(np.array([model.children_[:,0], model.children_[:,1], model.distances_]))
        #print('Computing h0', cm, i, j, len(X_))
        h0sr = sr._linkage_to_stable_rank(Z, contour = sr.standard_contour(), w_p = inf, w_q = 1, reduced = True)
        h0sr_test[cm].append([h0sr, i[0]])
        # get bc (how? just send distance matrix?)
        # get h1sr from bc
        # Need to encode the knn connectivity into d_
        knn_array = knn_graph.toarray()
        np.fill_diagonal(knn_array, 1)
        distance_full = spatial.distance.pdist(X_[['x', 'y']], "euclidean")
        distance_ref = np.max(distance_full)
        d_ = spatial.distance.squareform(distance_full)
        d_[np.where(knn_array == 0)] = distance_ref
        
        bc = sr._d_to_bc(d_, maxdim=1, thresh=inf, coeff=2, reduced=True)
        if len(h1sr_test) < len(X_test.index.unique()):
            #print('Computing h1')
            h1sr = sr.bc_to_sr(bc, degree="H1")
            h1sr_test.append([h1sr, i[0]])
    h0_kernel_test[cm] = np.asarray([[f[0].dot(g[0]) for g in h0sr_train[cm]] for f in h0sr_test[cm]])
h1_kernel_test = np.asarray([[f[0].dot(g[0]) for g in h1sr_train] for f in h1sr_test])

In [ ]:
h0_kernel_train ={}
h0_kernel_test ={}
start = timer()    
for cm in clustering_methods:
    h0_kernel_train[cm] = np.asarray([[f.dot(g) for g in h0sr_train[cm]] for f in h0sr_train[cm]])
    h0_kernel_test[cm] = np.asarray([[f.dot(g) for g in h0sr_train[cm]] for f in h0sr_test[cm]])
    
h1_kernel_train = np.asarray([[f.dot(g) for g in h1sr_train] for f in h1sr_train])
h1_kernel_test = np.asarray([[f.dot(g) for g in h1sr_train] for f in h1sr_test])

In [ ]:
plt.figure(figsize=(10,7))
for i,f in zip(X_train.index.unique(), h0sr_train["single"]):
    i = i[0]
    if i == 'TEAM_UFFE':
        color = 'blue'
    elif i == 'TEAM_MAGDA':
        color = 'red'
    f.plot(color=color, linewidth=1)
    plt.xlabel('Distance threshold', fontsize = 24)
    plt.ylabel('Rank of homology', fontsize = 24)
    plt.savefig('images/h0_sr_single_' + distance + '.png')

In [ ]:
plt.figure(figsize=(10,7))
for i,f in zip(X_train.index.unique(), h0sr_train["average"]):
    i = i[0]
    if i == 'TEAM_UFFE':
        color = 'blue'
    elif i == 'TEAM_MAGDA':
        color = 'red'
    f.plot(color=color, linewidth=1)
    plt.xlabel('Distance threshold', fontsize = 24)
    plt.ylabel('Rank of homology', fontsize = 24)
    plt.savefig('images/h0_sr_average_' + distance + '.png')

In [ ]:
plt.figure(figsize=(10,7))
for i,f in zip(X_train.index.unique(), h0sr_train["complete"]):
    i = i[0]
    if i == 'TEAM_UFFE':
        color = 'blue'
    elif i == 'TEAM_MAGDA':
        color = 'red'
    f.plot(color=color, linewidth=1)
    plt.xlabel('Distance threshold', fontsize = 24)
    plt.ylabel('Rank of homology', fontsize = 24)
    plt.savefig('images/h0_sr_complete_' + distance + '.png')

In [ ]:
plt.figure(figsize=(10,7))
for i,f in zip(X_train.index.unique(), h0sr_train["ward"]):
    i = i[0]
    if i == 'TEAM_UFFE':
        color = 'blue'
    elif i == 'TEAM_MAGDA':
        color = 'red'
    f.plot(color=color, linewidth=1)
    plt.xlabel('Distance threshold', fontsize = 24)
    plt.ylabel('Rank of homology', fontsize = 24)
    plt.savefig('images/h0_sr_ward_' + distance + '.png')

In [ ]:
plt.figure(figsize=(10,7))
for i,f in zip(X_train.index.unique(), h1sr_train):
    i = i[0]
    if i == 'TEAM_UFFE':
        color = 'blue'
    elif i == 'TEAM_MAGDA':
        color = 'red'
    f.plot(color=color, linewidth=1)
    plt.xlabel('Distance threshold', fontsize = 24)
    plt.ylabel('Rank of homology', fontsize = 24)
    plt.savefig('images/h1_sr_' + distance + '.png')

In [ ]:
prediction = {}
a_score = {}
for cm in clustering_methods:
    param_grid = {
    "C": np.linspace(0.01,0.2,20),
    }
    svc = SVC(kernel='precomputed')
    search = GridSearchCV(svc, param_grid, n_jobs=2, verbose = 1)
    search.fit(h0_kernel_train[cm], X_train.index.unique().get_level_values(0))
    #search.fit(h1_kernel_train, X_train.index.unique().get_level_values(0))
    print("Best parameter (CV score=%0.3f):" % search.best_score_)
    print(search.best_params_)
    prediction[cm] = search.predict(h0_kernel_test[cm])
    #prediction[cm] = search.predict(h1_kernel_test)
    a_score[cm] = accuracy_score(X_test.index.unique().get_level_values(0), prediction[cm])
    print(cm+": ", a_score[cm])

In [ ]:
prediction = {}
a_score = {}
for cm in clustering_methods:
    param_grid = {
    "C": np.linspace(0.01,0.2,20),
    }
    svc = SVC(kernel='precomputed')
    search = GridSearchCV(svc, param_grid, n_jobs=2, verbose = 1)
    #search.fit(h0_kernel_train[cm], X_train.index.unique().get_level_values(0))
    search.fit(h1_kernel_train, X_train.index.unique().get_level_values(0))
    print("Best parameter (CV score=%0.3f):" % search.best_score_)
    print(search.best_params_)
    #prediction[cm] = search.predict(h0_kernel_test[cm])
    prediction[cm] = search.predict(h1_kernel_test)
    a_score[cm] = accuracy_score(X_test.index.unique().get_level_values(0), prediction[cm])
    print(cm+": ", a_score[cm])

In [ ]:
composed_kernel_train = {}
composed_kernel_test = {}
for cm in clustering_methods:
    composed_kernel_train[cm] = np.add(h0_kernel_train[cm], h1_kernel_train)
    composed_kernel_test[cm] = np.add(h0_kernel_test[cm], h1_kernel_test)

In [ ]:
composed_kernel_train = {}
composed_kernel_test = {}
for cm in clustering_methods:
    composed_kernel_train[cm] = np.max(np.array([h0_kernel_train[cm], h1_kernel_train]), axis = 0)
    composed_kernel_test[cm] = np.max(np.array([h0_kernel_test[cm], h1_kernel_test]), axis = 0)

In [ ]:
composed_kernel_train = {}
composed_kernel_test = {}
for cm in clustering_methods:
    composed_kernel_train[cm] = np.min(np.array([h0_kernel_train[cm], h1_kernel_train]), axis = 0)
    composed_kernel_test[cm] = np.min(np.array([h0_kernel_test[cm], h1_kernel_test]), axis = 0)

In [ ]:
composed_kernel_train = {}
composed_kernel_test = {}
for cm in clustering_methods:
    composed_kernel_train[cm] = np.linalg.norm(np.array([h0_kernel_train[cm], h1_kernel_train]), axis = 0, ord = 2)
    composed_kernel_test[cm] = np.linalg.norm(np.array([h0_kernel_test[cm], h1_kernel_test]), axis = 0, ord = 2)

In [ ]:
prediction = {}
a_score = {}
for cm in clustering_methods:
    param_grid = {
    "C": np.linspace(0.01,0.2,20),
    }
    svc = SVC(kernel='precomputed')
    search = GridSearchCV(svc, param_grid, n_jobs=2, verbose = 1)
    search.fit(composed_kernel_train[cm], X_train.index.unique().get_level_values(0))
    print("Best parameter (CV score=%0.3f):" % search.best_score_)
    print(search.best_params_)
    #prediction[cm] = search.predict(h0_kernel_test[cm])
    prediction[cm] = search.predict(composed_kernel_test[cm],)
    a_score[cm] = accuracy_score(X_test.index.unique().get_level_values(0), prediction[cm])
    print(cm+": ", a_score[cm])